In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

In [3]:
wine_data = "Datasets/winemag-data-130k-v2.csv"

In [4]:
wine_data_df = pd.read_csv(wine_data)

In [5]:
wine_data_df.head(2)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos


In [6]:
wine_data_df.columns

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')

In [7]:
reduced_wine_data_df = wine_data_df[["country", "points", "price", "variety", "province", "winery"]]

In [8]:
drop_na_df = reduced_wine_data_df.dropna()

In [9]:
drop_na_df.count()

country     120915
points      120915
price       120915
variety     120915
province    120915
winery      120915
dtype: int64

In [13]:
#groupbys for dataset
country_group = drop_na_df.groupby("country")
province_group = drop_na_df.groupby("province")
winery_group = drop_na_df.groupby("winery")
country_by_province = drop_na_df.groupby(["country", "province"])
country_by_province.mean()



points      price
country   province                                 
Argentina Mendoza Province     86.826410  25.053317
          Other                86.003774  21.203774
Armenia   Armenia              87.500000  14.500000
Australia Australia Other      85.483051  12.427966
          New South Wales      87.694118  25.623529
          South Australia      89.056349  38.811420
          Tasmania             89.071429  36.523810
          Victoria             88.869841  42.606349
          Western Australia    88.915789  33.578947
Austria   Austria              87.090909  29.045455
          Burgenland           89.647266  28.984127
          Carnuntum            90.416667  28.208333
          Eisenberg            91.187500  35.375000
          Kamptal              91.396450  33.869822
          Kremstal             90.863014  30.071918
          Leithaberg           91.441176  44.294118
          Mittelburgenland     88.789474  25.894737
          Neusiedlersee        88.636364  30.909091
          Niederösterreich     89.095634  23.411642
          Steiermark           87.571429  19.571429
          Südburgenland        94.000000  57.000000
          Südoststeiermark     89.222222  29.740741
          Südsteiermark        89.820513  31.089744
          Thermenregion        90.214953  32.308411
          Traisental           91.255814  32.162791
          Vienna               90.448276  26.568966
          Wachau               91.665584  45.902597
          Wagram               90.854839  28.354839
          Wagram-Donauland     89.558824  30.323529
          Weinland Österreich  88.000000  22.437500
...                                  ...        ...
US        Hawaii               87.000000  28.000000
          Idaho                86.605263  20.768421
          Illinois             84.500000  32.333333
          Iowa                 83.000000  15.500000
          Kentucky             83.000000  35.000000
          Massachusetts        84.857143  21.285714
          Michigan             86.153153  32.432432
          Missouri             83.333333  17.333333
          Nevada               84.250000  37.750000
          New Jersey           84.000000  25.500000
          New Mexico           83.955556  22.422222
          New York             87.181241  22.826233
          North Carolina       84.565217  21.000000
          Ohio                 84.083333  13.750000
          Oregon               89.053741  36.545064
          Pennsylvania         84.555556  23.333333
          Rhode Island         90.000000  25.000000
          Texas                85.215054  25.602151
          Vermont              84.333333  31.666667
          Virginia             85.619481  27.012987
          Washington           88.951416  32.436444
          Washington-Oregon    88.428571  17.857143
Ukraine   Ukraine              84.071429   9.214286
Uruguay   Atlantida            86.400000  22.000000
          Canelones            86.651163  23.511628
          Juanico              86.333333  48.583333
          Montevideo           88.272727  26.090909
          Progreso             86.818182  24.272727
          San Jose             84.000000  30.000000
          Uruguay              86.833333  22.083333

[422 rows x 2 columns]